In [ ]:
import torch

from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu' #GPU 사용이 가능하다면 GPU를 사용하고 아니면 CPU를 사용한다.

In [ ]:
# parameters
learning_rate = 0.001 #학습율, 하이퍼파라미터로 학습하는 것이 아니고 입력하는 값
epochs = 1            # 전체 데이터를 1번 학습한다.
batch_size = 100      # 데이터 학습시 데이터의 단위(100개 씩 학습함)
drop_out = 0.3        # 1 epoch 학습할 때 0.3프로 연결을 끊는다. 정규화하는 방법 중의 하나이다.

In [ ]:
# MNIST dataset
training_data = datasets.MNIST(root='MNIST_data/', #데이터의 위치
                          train=True,
                          transform=transforms.ToTensor(), #이미지를 tensor에 맞게 조정
                          download=True) #데이터를 root위치에 다운로드한다. 이미 존재하는 경우 다시 다운하지 않는다.

test_data = datasets.MNIST(root='MNIST_data/',
                         train=False, 
                         transform=transforms.ToTensor(),
                         download=True)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting MNIST_data/MNIST/raw/train-images-idx3-ubyte.gz to MNIST_data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting MNIST_data/MNIST/raw/train-labels-idx1-ubyte.gz to MNIST_data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting MNIST_data/MNIST/raw/t10k-images-idx3-ubyte.gz to MNIST_data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting MNIST_data/MNIST/raw/t10k-labels-idx1-ubyte.gz to MNIST_data/MNIST/raw



In [ ]:
train_dataloader = DataLoader(training_data, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=batch_size, shuffle=True)                                    

In [ ]:
class CNN(nn.Module):
    
    def __init__(self):
        super(CNN, self).__init__()
        #첫번째 layer 생성
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size = 3, stride =1, padding = 1), # 입력 채널 개수(흑백이미지), 출력 채널 개수, 건너 띄기 없음, same padding
            nn.ReLU(), #activation function. (비선형함수, 음수는 0으로 바꾼다)
            nn.MaxPool2d(kernel_size = 2, stride = 2)) #28x28 -> 14x14x32로 사이즈가 줄어든다
        
        #두 번째 layer 생성
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, stride =1, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2)) #14x14x32 -> 7x7x64로 사이즈가 줄어든다
        
        #fully connected layer 생성
        self.fc = nn.Linear(7 * 7 * 64, 10, bias = True) # 행렬 데이터를 벡터로 바꿔를 입력하여 10개(0~9)로 출력함
        nn.init.xavier_uniform_(self.fc.weight)
   
   #모델 설계 후 데이터셋이 layer들을 통과할 수 있게 함.
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1) 
        out = self.fc(out)
        return out

In [ ]:
model = CNN().to(device)

In [ ]:
#cost & loss 명시하기 & optimizer
# pytorch는 softmax가 포함이 되어 있기 때문에 생략
criterion = nn.CrossEntropyLoss().to(device)    
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate) #gradient descent할 때 adam 활용

In [ ]:
total_batch = len(train_dataloader)
print(total_batch) # total batch값 600출력. (6만개의 데이터가 batch_size인 100으로 나뉜 값)
print('Learning started. It takes sometime.')
for epoch in range(epochs):
    avg_cost = 0

    for X, Y in train_dataloader:
        X = X.to(device) # batch_size(100), 채널(1=흑백), 28 X 28 사이즈의 이미지 데이터를 입력함
        Y = Y.to(device)

        optimizer.zero_grad()            # optimizer 변수에 포함시킨 매개 변수(weight)들의 기울기(Gradient)를 0으로 초기화
        hypothesis = model(X)            # 가설 설정
        cost = criterion(hypothesis, Y)  # 비용 설정
        cost.backward()                  # 역전파
        optimizer.step()                 # 학습을 통해 계산한 weight, bias, gradient를 최적화함수(optimizer)에에 반영

        avg_cost += cost / total_batch

    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))

print('Learning Finished!')

600
Learning started. It takes sometime.
[Epoch:    1] cost = 0.218491718
Learning Finished!


In [ ]:
with torch.no_grad(): #gradient 비활성화
    X_test = test_data.test_data.view(len(test_data), 1, 28, 28).float().to(device)
    Y_test = test_data.test_labels.to(device)

    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test # 가장 높은 확율을 값을 골라서 정답과 비교
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())

Accuracy: 0.9789999723434448


/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:80: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:70: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")
